In [0]:
from copy import copy 
from collections import namedtuple
from functools import reduce

import math


Item = namedtuple('Item', ['cost', 'weight'])

Обозначим через $opt(k,c)$ минимальный объем, необходимый для того, чтобы уложить предметы с номерами, не превосходящими $k ∈ [1..n]$, общей стоимостью не менее $c$.

1. Положим $opt(0,0) = 0,$  $opt(0,p) = ∞, p > 0$

2. В цикле по $k$ от 1 до $n$ вычислим $opt(k,c)$ для каждого $c$ от $1$ до $\sum_{i∈[1..n]} c_i$:   
      $opt(k + 1, c) = min${ $opt(k, c),$  $opt(k, c − c_{k+1}) + w_{k+1}$ }   

3. В качестве ответа вернем $max$ {$c$  |  $opt(n,c) ≤ C$}

In [0]:
def launch_pseudo_poly(items, min_cost, max_weight):
    n, inf = len(items), max_weight+1
    max_cost = reduce(lambda cost, item: cost + item.cost, items, 0)
    opt = [[inf if k==0 and c>0 else 0 for c in range(max_cost+1)] 
                       for k in range(n+1)]
    for k in range(0, n):
        for c in range(1, max_cost+1):
            opt[k+1][c] = min(opt[k][c], opt[k][max(0, c-items[k].cost)] + items[k].weight)
    print(opt)
    return max([0]+list(filter(lambda c: opt[n][c]<=max_weight, range(min_cost, max_cost+1))))

In [0]:
print(launch_pseudo_poly([Item(cost=1, weight=2), Item(cost=2, weight=3)], 3, 5))



---



In [0]:
class Item:

  def __init__(self, weight=0, value=0):
    self.weight = weight
    self.cost = cost

In [0]:
item_compare = lambda item0, item1: item0.value < item1.value

In [0]:
class ItemSet:

  def __init__(self, item_set=None):
    self.item_set = set() if item_set is None else item_set

  @property
  def max_cost(self):
    return max(map(lambda item: item.cost, self.item_set))

  @property
  def total_cost(self):
    return sum(map(lambda item: item.cost, self.item_set))
  
  @property
  def items_count(self):
    return len(self.item_set)

  def append(self, item: Item):
    self.item_set.add(item)

In [0]:
from copy import deepcopy


class KnapsackHandler(ItemSet):

  def __init__(self, items_set: set, capacity, epsilon):
    super(ItemSet, self).__init__(items_set)
    self.capacity = capacity
    self.epsilon = epsilon

  def run_poly_algo(self):
    total_value = self.total_cost
    temp = [-1] * (total_value + 1)
    partial_weights = [deepcopy(temp) for _ in range(self.items_count + 1)]
    

In [0]:
#include <iostream>
#include <vector>
#include <limits>
#include <algorithm>
#include <map>


using std::vector;
using std::map;


struct Item {
    double weight;
    double value;

    Item() {
        weight = 0.0;
        value = 0.0;
    }

    Item(double _weight, double _value) {
        weight = _weight;
        value = _value;
    }
};


static bool item_compare(Item & first, Item & second) {
    return (first.value < second.value);
}


struct ItemSet {
    vector<Item> set;

    ItemSet() {
        set.resize(0);
    }

    ItemSet(vector<Item> _set) {
        set  =_set;
    }

    int sum_value() {
        double sum = 0;

        for (Item item : set) {
            sum += item.value;
        }

        return sum;
    }

    int sum_weight() {
        double sum = 0;

        for (Item item : set) {
            sum += item.weight;
        }

        return sum;
    }

    void append(Item item) {
        set.push_back(item);
    }
};


class KnapsackHandler {
private:
    ItemSet items;
    double capacity;
    int items_count;
    double max_value;
    double epsilon;

public:
    KnapsackHandler(ItemSet & _items, double _capacity, double _epsilon) {
        items = _items;
        capacity = _capacity;
        items_count = items.set.size();
        max_value = std::max_element(items.set.begin(), items.set.end(), item_compare)->value;
        epsilon = _epsilon;
    }

    vector<bool> pseudopolynomial_knapsack() {
        double total_value = items.sum_value();

        vector<double> temp(total_value + 1, -1);
        vector<vector<double>> partial_weights(items_count + 1, temp);

        vector<bool> temp_set(items_count);
        vector<vector<bool>> temp_set_line(total_value + 1, temp_set);
        vector<vector<vector<bool>>> partial_sets(items_count + 1, temp_set_line);

        for (int value = 1; value <= items_count; ++value) {
            partial_weights[0][value] = -1;
        }
        partial_weights[0][0] = 0;

        for (int j = 1; j <= items_count; ++j) {
			std::cout << j << " ";
            for (int k = 0; k <= total_value; ++k) {
                Item & cur_item = items.set[j - 1];
                double previous_value = partial_weights[j - 1][k];
                vector<bool> & previous_set = partial_sets[j - 1][k];

                if (cur_item.value > k) {
                    partial_weights[j][k] = previous_value;
                    partial_sets[j][k] = previous_set;
                    continue;
                }

                double test_value = partial_weights[j - 1][k - cur_item.value];
                vector<bool> & test_set = partial_sets[j - 1][k - cur_item.value];

                if (test_value == -1) {
                    partial_weights[j][k] = previous_value;
                    partial_sets[j][k] = previous_set;
                    continue;
                }

                double upper_bound = previous_value == -1 ? capacity : std::min(capacity, previous_value);

                if (test_value + cur_item.weight <= upper_bound) {
                    partial_weights[j][k] = test_value + cur_item.weight;
                    partial_sets[j][k] = test_set;
                    partial_sets[j][k][j - 1] = true;
                } else {
                    partial_weights[j][k] = previous_value;
                    partial_sets[j][k] = previous_set;
                }
            }
        }

        double optimal_value = 0;
        for (int k = 0; k <= total_value; ++k) {
            if (partial_weights[items_count][k] != -1) {
                optimal_value = k;
            }
        }

        return partial_sets[items_count][optimal_value];
    }

    vector<bool> polinomial_knapsack_approximation() {
        double k_epsilon = max_value * epsilon / items_count;

        for (int i = 0; i < items_count; ++i) {
            items.set[i].value = int(items.set[i].value / k_epsilon);
        }

        auto optimal_set = pseudopolynomial_knapsack();

        return optimal_set;
    }
};


void input(ItemSet & items, double & capacity, double & epsilon) {
    int items_count;
    std::cin >> items_count;

    vector<double> weights, values;
    weights.resize(items_count);
    values.resize(items_count);

    for (int i = 0; i < items_count; ++i) {
        std::cin >> weights[i];
    }

    for (int i = 0; i < items_count; ++i) {
        std::cin >> values[i];
        items.set.push_back(Item(weights[i], values[i]));
    }

    std::cin >> capacity;
    std::cin >> epsilon;
}


void output(const vector<bool> optimal_set, const ItemSet & items) {
    bool isEmpty = true;

    for (int i = 0; i < optimal_set.size(); ++i) {
        if (optimal_set[i] == true) {
            isEmpty = false;
            break;
        }
    }

    if (isEmpty) {
        std::cout << "The optimal knapsack doesn't exist" << std::endl;
        return;
    }

    std::cout << "The optimal knapsack includes:" << std::endl;
    for (int i = 0; i < optimal_set.size(); ++i) {
        if (optimal_set[i]) {
            std::cout << "Value: " << items.set[i].value << ", weight: " << items.set[i].weight << std::endl;
        }
    }
}


int main() {
    ItemSet items;
    double capacity, epsilon;

    input(items, capacity, epsilon);

    KnapsackHandler knapsack_handler(items, capacity, epsilon);

    output(knapsack_handler.polinomial_knapsack_approximation(), items);

    return 0;
}


SyntaxError: ignored

In [0]:
#include <algorithm>
#include <chrono>
#include <functional>
#include <string>
#include <iomanip>
#include <iostream>
#include <limits>
#include <map>
#include <vector>

template<typename T>
struct Item {
  T weight;
  T value;
  int64_t round_value;
};

// Ð¡Ð¾Ð·Ð´Ð°Ñ‘Ñ‚ Ð½Ð°Ð±Ð¾Ñ€ Ð¸Ð· Ð¿Ñ€ÐµÐ´Ð¼ÐµÑ‚Ð¾Ð² Ñ Ð²ÐµÑÐ°Ð¼Ð¸ Ð¸ ÑÑ‚Ð¾Ð¸Ð¼Ð¾ÑÑ‚ÑÐ¼Ð¸ Ð²Ð¸Ð´Ð° C * 2^k
template<typename T>
std::vector<Item<T>> GeneratePowerTwoItemsSet(int size, T minumim_value) {
  std::vector<Item<T>> items;
  T current_value = minumim_value;
  for (int i = 0; i < size; ++i) {
    items.push_back(Item<T>{current_value, current_value, 0});
    current_value *= 2;
  }
  return items;
}

// Ð¡Ð¾Ð·Ð´Ð°Ñ‘Ñ‚ Ð½Ð°Ð±Ð¾Ñ€ Ð¸Ð· Ð¿Ñ€ÐµÐ´Ð¼ÐµÑ‚Ð¾Ð² Ñ Ð²ÐµÑÐ°Ð¼Ð¸ Ð¸ ÑÑ‚Ð¾Ð¸Ð¼Ð¾ÑÑ‚ÑÐ¼Ð¸ 1, 2, ..., k
template<typename T>
std::vector<Item<T>> GenerateItemsSeqSet(int size) {
  std::vector<Item<T>> items;
  for (int i = 0; i < size; ++i) {
    items.push_back(Item<T>{static_cast<long double>(i + 1),
      static_cast<long double>(i + 1), 0});
  }
  return items;
}

// Ð˜Ñ‰ÐµÑ‚ Ñ€ÐµÑˆÐµÐ½Ð¸Ðµ, Ð¿ÐµÑ€ÐµÐ±Ð¸Ñ€Ð°Ñ Ð²ÑÐµ Ð½Ð°Ð±Ð¾Ñ€Ñ‹ Ð¿Ñ€ÐµÐ´Ð¼ÐµÑ‚Ð¾Ð²
template<typename T>
std::vector<Item<T>> FindSolutionByBruteforce(const std::vector<Item<T>> &items, T max_weight) {
  T maximum_value = 0;
  uint64_t maximum_set = 0;
  
  for (uint64_t current_set = 0; current_set < (1 << items.size()); ++current_set) {
    T current_value = 0;
    T current_weight = 0;
    for (int i = 0; i < items.size() && current_weight <= max_weight; ++i) {
      if ((current_set >> i) & 1) {
        current_value += items[i].value;
        current_weight += items[i].weight;
      }
    }
    if (current_weight <= max_weight && current_value > maximum_value) {
      maximum_value = current_value;
      maximum_set = current_set;
    }
  }
  
  std::vector<Item<T>> maximum_set_items;
  for (int i = 0; i < items.size(); ++i) {
    if ((maximum_set >> i) & 1) {
      maximum_set_items.push_back(items[i]);
    }
  }
  return maximum_set_items;
}

// Ð’ÑÐ¿Ð¾Ð¼Ð¾Ð³Ð°Ñ‚ÐµÐ»ÑŒÐ½Ð°Ñ ÑÑ‚Ñ€ÑƒÐºÑ‚ÑƒÑ€Ð° Ð´Ð»Ñ Ñ€ÐµÑˆÐµÐ½Ð¸Ñ Ð¼ÐµÑ‚Ð¾Ð´Ð¾Ð¼ Ð´Ð¸Ð½Ð°Ð¼Ð¸Ñ‡ÐµÑÐºÐ¾Ð³Ð¾ Ð¿Ñ€Ð¾Ð³Ñ€Ð°Ð¼Ð¼Ð¸Ñ€Ð¾Ð²Ð°Ð½Ð¸Ñ
template<typename T>
struct DynamicState {
  T minumim_weight;
  T value;
  int last_item;
};

// Ð˜Ñ‰ÐµÑ‚ Ñ€ÐµÑˆÐµÐ½Ð¸Ðµ Ð¼ÐµÑ‚Ð¾Ð´Ð¾Ð¼ Ð´Ð¸Ð½Ð°Ð¼Ð¸Ñ‡ÐµÑÐºÐ¾Ð³Ð¾ Ð¿Ñ€Ð¾Ð³Ñ€Ð°Ð¼Ð¼Ð¸Ñ€Ð¾Ð²Ð°Ð½Ð¸Ñ
// BÑÐ¿Ð¾Ð»ÑŒÐ·ÑƒÑŽÑ‚ÑÑ Ð·Ð½Ð°Ñ‡ÐµÐ½Ð¸Ñ round_value
template<typename T>
std::vector<DynamicState<T>> FindSolutionByDynamic(std::vector<Item<T>> items, T max_weight,
                                                   int64_t max_value = 0) {
  if (max_value == 0) {
    for (const auto &item : items) {
      max_value += item.round_value;
    }
  }
  
  std::vector<DynamicState<T>> states(
                                      max_value + 1, DynamicState<T>{std::numeric_limits<T>::max(), 0, 0});
  states[0] = DynamicState<T>{0, 0, 0};
  for (int i = 0; i < items.size(); ++i) {
    for (int64_t value = max_value; value >= items[i].round_value; --value) {
      if (items[i].weight + states[value - items[i].round_value].minumim_weight <
          states[value].minumim_weight) {
        states[value] = DynamicState<T>{
          items[i].weight + states[value - items[i].round_value].minumim_weight,
          items[i].value + states[value - items[i].round_value].value, i};
      }
    }
  }
  return states;
}

// Ð—Ð°Ð¼ÐµÐ½ÑÐµÑ‚ ÑÑ‚Ð¾Ð¸Ð¼Ð¾ÑÑ‚Ð¸ p Ð½Ð° Ñ†ÐµÐ»Ñ‹Ðµ [p / delta]
template<typename T>
void RoundValues(T delta, std::vector<Item<T>> *items) {
  for (auto &item : *items) {
    item.round_value = item.value / delta;
  }
}

// Ð˜Ñ‰ÐµÑ‚ Ð¶Ð°Ð´Ð½Ð¾Ðµ Ñ€ÐµÑˆÐµÐ½Ð¸Ðµ Ð¿Ð¾ Ð½Ð°Ð±Ð¾Ñ€Ñƒ Ð¿Ñ€ÐµÐ´Ð¼ÐµÑ‚Ð¾Ð², Ð¾Ñ‚ÑÐ¾Ñ€Ñ‚Ð¸Ñ€Ð¾Ð²Ð°Ð½Ð¾Ð¼Ñƒ Ð¿Ð¾ Ð¿Ð»Ð¾Ñ‚Ð½Ð¾ÑÑ‚Ð¸
template<typename T>
void FindGreedySolution(
                        const std::vector<Item<T>> &items, T max_weight,
                        int *greedy_solution_size, T *greedy_solution_weight, T *greedy_solution_value) {
  *greedy_solution_size = 0;
  *greedy_solution_weight = *greedy_solution_value = 0;
  while (*greedy_solution_size < items.size() &&
         *greedy_solution_weight + items[*greedy_solution_size].weight < max_weight) {
    *greedy_solution_weight += items[*greedy_solution_size].weight;
    *greedy_solution_value += items[*greedy_solution_size].value;
    ++*greedy_solution_size;
  }
}

// Ð˜Ñ‰ÐµÑ‚ Ð¿Ñ€Ð¸Ð±Ð»ÐµÐ¶ÐµÐ½Ð½Ð¾Ðµ Ñ€ÐµÑˆÐµÐ½Ð¸Ðµ Ð°Ð»Ð³Ð¾Ñ€Ð¸Ñ‚Ð¼Ð¾Ð¼ Ð˜Ð±Ð°Ñ€Ñ€Ñ‹ Ð¸ ÐšÐ¸Ð¼Ð°
template<typename T>
std::vector<Item<T>> FindApproximateSolutionByIbarraKim(
                                                        std::vector<Item<T>> items, T max_weight, T precision) {
  
  // Ð’Ñ‹ÐºÐ¸Ð½ÐµÐ¼ ÑÐ»Ð¸ÑˆÐºÐ¾Ð¼ Ñ‚ÑÐ¶Ñ‘Ð»Ñ‹Ðµ Ð¿Ñ€ÐµÐ´Ð¼ÐµÑ‚Ñ‹
  auto items_end = std::remove_if(items.begin(), items.end(),
                                  [max_weight](const Item<T> &item) { return item.weight > max_weight; });
  items.resize(std::distance(items.begin(), items_end));
  
  // ÐžÑ‚ÑÐ¾Ñ€Ñ‚Ð¸Ñ€ÑƒÐµÐ¼ Ð¿Ñ€ÐµÐ´Ð¼ÐµÑ‚Ñ‹ Ð¿Ð¾ Ð¿Ð»Ð¾Ñ‚Ð½Ð¾ÑÑ‚Ð¸
  std::sort(items.begin(), items.end(), [](const Item<T> &first, const Item<T> &second) {
    return first.value / first.weight > second.value / second.weight;
  });
  
  // ÐÐ°Ð¹Ð´Ñ‘Ð¼ Ñ€Ð°Ð·Ð¼ÐµÑ€ Ð¶Ð°Ð´Ð½Ð¾Ð³Ð¾ Ñ€ÐµÑˆÐµÐ½Ð¸Ñ
  int greedy_solution_size = 0;
  T greedy_solution_weight = 0;
  T greedy_solution_value = 0;
  FindGreedySolution(items, max_weight, &greedy_solution_size,
                     &greedy_solution_weight, &greedy_solution_value);
  
  if (greedy_solution_size == items.size()) {
    return items;
  }
  double P_g = greedy_solution_value + items[greedy_solution_size].value;
  
  // Ð Ð°Ð·Ð¾Ð±ÑŒÑ‘Ð¼ Ð¿Ñ€ÐµÐ´Ð¼ÐµÑ‚Ñ‹ Ð½Ð° Ð´ÐµÑˆÑ‘Ð²Ñ‹Ðµ Ð¸ Ð´Ð¾Ñ€Ð¾Ð³Ð¸Ðµ
  auto items_middle = std::stable_partition(items.begin(), items.end(),
                                            [P_g, precision](const Item<T> &item) {
                                              return item.value >= precision * P_g;
                                            });
  std::vector<Item<T>> expensive_items(items.begin(), items_middle);
  std::vector<Item<T>> cheap_items(items_middle, items.end());
  
  
  // ÐžÐºÑ€ÑƒÐ³Ð»Ð¸Ð¼ ÑÑ‚Ð¾Ð¸Ð¼Ð¾ÑÑ‚Ð¸ Ð´Ð¾Ñ€Ð¾Ð³Ð¸Ñ… Ð¿Ñ€ÐµÐ´Ð¼ÐµÑ‚Ð¾Ð²
  RoundValues(precision * precision * P_g, &expensive_items);
  
  std::vector<DynamicState<T>> dynamic_states =
  FindSolutionByDynamic(expensive_items, max_weight, 2 / precision / precision + 1);
  
  T max_value = 0;
  int max_value_exp_rounded_value = 0;
  int max_value_greedy_solution_size = 0;
  for (int rounded_value = 0; rounded_value < dynamic_states.size(); ++rounded_value) {
    if (dynamic_states[rounded_value].minumim_weight > max_weight) {
      continue;
    }
    int local_greedy_solution_size = 0;
    T local_greedy_solution_weight = 0;
    T local_greedy_solution_value = 0;
    FindGreedySolution(
                       cheap_items, max_weight - dynamic_states[rounded_value].minumim_weight,
                       &local_greedy_solution_size, &local_greedy_solution_weight,
                       &local_greedy_solution_value);
    
    if (local_greedy_solution_value + dynamic_states[rounded_value].value > max_value) {
      max_value = local_greedy_solution_value + dynamic_states[rounded_value].value;
      max_value_exp_rounded_value = rounded_value;
      max_value_greedy_solution_size = local_greedy_solution_size;
    }
  }
  
  std::vector<Item<T>> maximum_set_items;
  int current_value = max_value_exp_rounded_value;
  while (current_value > 0) {
    maximum_set_items.push_back(expensive_items[dynamic_states[current_value].last_item]);
    current_value -= maximum_set_items.back().round_value;
  }
  std::reverse(maximum_set_items.begin(), maximum_set_items.end());
  for (int i = 0; i < max_value_greedy_solution_size; ++i) {
    maximum_set_items.push_back(cheap_items[i]);
  }
  
  return maximum_set_items;
}

// Ð’Ñ‹Ð²Ð¾Ð´Ð¸Ñ‚ Ð¼Ð½Ð¾Ð¶ÐµÑÑ‚Ð²Ð¾, Ð½Ð°Ð¹Ð´ÐµÐ½Ð½Ð¾Ðµ Ñ€ÐµÑˆÐµÐ½Ð¸ÐµÐ¼
template<typename T>
void PrintSet(const std::vector<Item<T>> &items) {
  T set_value = 0;
  T set_weight = 0;
  for (const auto &item : items) {
    set_value += item.value;
    set_weight += item.weight;
  }
  
  std::cout << std::setprecision(18);
  std::cout << "General set price: " << set_value << ", general set weight: " << set_weight << "\n";
  for (const auto &item : items) {
    std::cout << "item: " << "price " << item.value <<
    ", rounded value " << item.round_value << ", weight " << item.weight << "\n";
  }
  std::cout << "\n";
}

// Ð’Ñ‹Ð·Ñ‹Ð²Ð°ÐµÑ‚ Ñ„ÑƒÐ½ÐºÑ†Ð¸ÑŽ Ð¸ Ð·Ð°Ð¼ÐµÑ€ÑÐµÑ‚ Ð²Ñ€ÐµÐ¼Ñ Ñ€Ð°Ð±Ð¾Ñ‚Ñ‹
template<typename F>
auto RunAndMeasureTime(F function, double *working_time) {
  std::chrono::steady_clock::time_point start_time = std::chrono::steady_clock::now();
  auto result = function();
  std::chrono::steady_clock::time_point end_time = std::chrono::steady_clock::now();
  *working_time = std::chrono::duration_cast<std::chrono::microseconds>(
                                                                        end_time - start_time).count();
  return result;
}

// Ð—Ð°Ð¼ÐµÑ€ÑÐµÑ‚ Ð²Ñ€ÐµÐ¼Ñ Ñ€Ð°Ð±Ð¾Ñ‚Ñ‹ Ñ€ÐµÑˆÐµÐ½Ð¸Ñ Ð½Ð° Ð´Ð°Ð½Ð½Ð¾Ð¼ Ð¼Ð½Ð¾Ð¶ÐµÑÑ‚Ð²Ðµ Ð½Ð° Ð´Ð°Ð½Ð½Ñ‹Ñ… Ð·Ð½Ð°Ñ‡ÐµÐ½Ð¸ÑÑ… Ñ‚Ð¾Ñ‡Ð½Ð¾ÑÑ‚Ð¸
template<typename T>
void TestOnItemsSet(const std::vector<Item<T>> &items, T max_weight,
                    const std::vector<T> &precisions, const std::string &comment) {
  std::cout << "running " << comment << "...\n";
  
  // {
  //     double bruteforce_time;
  //     auto bruteforce_solution =
  //         RunAndMeasureTime(
  //             std::bind(FindSolutionByBruteforce<T>, items, max_weight), &bruteforce_time);
  //     std::cout << "bruteforce time: " << bruteforce_time << "ms\n";
  //     PrintSet(bruteforce_solution);
  // }
  
  for (const auto &precision : precisions) {
    double dynamic_time;
    auto dynamic_solution = RunAndMeasureTime(
                                              std::bind(FindApproximateSolutionByIbarraKim<T>, items, max_weight, precision),
                                              &dynamic_time);
    std::cout << "precision: " << precision <<
    ", Ibarra Kim time: " << dynamic_time/1000 << "ms\n";
//    PrintSet(dynamic_solution);
  }
  
  std::cout << "\n\n";
}

int main() {
  std::vector<Item<double>> items = {
    Item<double>{5, 1, 0},
    Item<double>{2, 0.9, 0},
    Item<double>{3, 0.91, 0},
    Item<double>{1, 0.1, 0},
  };
  
  TestOnItemsSet(items, 4.001, {0.5, 0.1, 0.01}, "sample set");

  TestOnItemsSet<long double>(GeneratePowerTwoItemsSet<long double>(25, 1),
                              1000000.001, {0.5, 0.1, 1e-2, 1e-3}, "25 items 2pow set");
  TestOnItemsSet<long double>(GeneratePowerTwoItemsSet<long double>(27, 1),
                              1000000.001, {0.5, 0.1, 1e-2, 1e-3}, "27 items 2pow set");
  TestOnItemsSet<long double>(GeneratePowerTwoItemsSet<long double>(30, 1),
                              1000000.001, {0.5, 0.1, 1e-2, 1e-3}, "30 items 2pow set");

  TestOnItemsSet<long double>(GenerateItemsSeqSet<long double>(30),
                              400, {0.5, 0.1, 1e-2, 1e-3}, "30 items conseq set");

  TestOnItemsSet<long double>(GenerateItemsSeqSet<long double>(100),
                              450, {0.5, 0.1, 1e-2, 1e-3}, "100 items conseq set");
  
  TestOnItemsSet<long double>(GenerateItemsSeqSet<long double>(1000),
                              4500, {0.5, 0.1, 1e-2, 1e-3}, "1000 items conseq set");

  TestOnItemsSet<long double>(GenerateItemsSeqSet<long double>(10000),
                              450000, {0.5, 0.1, 1e-2}, "10000 items conseq set");
  
  return 0;
}




---



---



---



In [0]:
class Item(object):

  __slots__ = ['weight', 'value', 'round_value']

  def __init__(self, weight=0, value=0, round_value=0, *args, **kwargs):
    self.weight = weight
    self.value = value
    self.round_value = round_value

In [0]:
class DynamicState(object):
  
  __slots__ = ['min_weight', 'value', 'last_item']

  def __init__(self, min_weight=math.inf, value=0, last_item=-1, *args, **kwargs):
    self.min_weight = min_weight
    self.value = value
    self.last_item = last_item

In [0]:
class GreedySolution(object):
    
    __slots__ = ['size', 'weight', 'value']

    def __init__(self, size=0, weight=0, value=0, *args, **kwargs):
        self.size = size
        self.weight = weigth
        self.value = value

    def __iadd__(self, item: Item):
      self.size += 1
      self.weight = item.weigth
      self.value = item.value

GreedySolution(size=1, weight=2, value=3)


In [0]:
import math
import functools

from itertools import combinations, chain

In [0]:
all_indexes_subsets = lambda size: filter(
    lambda subset: len(subset) > 0,
    chain(*[combinations(range(size), i) for i in range(size+1)])
)

In [0]:
def solve_bruteforce(items, max_weight) -> tuple:
  total_value, total_weight = 0, math.inf
  for subset in all_indexes_subsets(len(items)):
    sub_total_value, sub_total_weight = 0, math.inf
    for index in subset:
      sub_total_value += items[index].value
      sub_total_weight += items[index].weight
    if sub_total_value > total_value:
      total_value, total_weight = sub_total_value, sub_total_weight
    elif sub_total_value == total_value and sub_total_weight < total_weight:
      sub_total_weight = total_weight
  return total_weight, total_value

In [0]:
def solve_dynamic(items, max_weight, max_value=0) -> list:
  if max_value == 0:
    max_value = functools.reduce(lambda total, item: total + item.round_value,
                                items)
  states = [DynamicState() for _ in range(max_value+1)]
  states[0].min_weight = 0
  for (index, item) in enumerate(items):
    for value in range(max_value, item.round_value, -1):
      if item.weight + states[value-item.round_value].min_weight < \
      states[value].min_weight:
        states[value] = DynamicState(
            item.weight + states[value-item.round_value].min_weight,
            item.value + states[value-item.round_value].value, index
        )
  return states

In [0]:
def solve_greedy(items, max_weight) -> GreedySolution:
  solution = GreedySolution()
  while solution.size < len(items) and \
  solution.weight + items[solution.size].weight < max_weigth:
    solution += items[solution.size]
  return solution

In [0]:
def solve_ibarra_kim(items, max_weight, precision) -> list:
  items = list(filter(lambda item: item.weight > max_weight, items))
  items = sorted(items, key=lambda item: -item.value/item.weight)
  solution = solve_greedy(item, max_weight)
  if solution.size == len(items):
    return items
  P_g = solution.value + items[solution.size].value
  threshold = lambda item: item.value >= precision*P_g
  antithreshold = lambda item: not threshold(item)
  expensive_items = list(filter(threshold, items))
  cheap_items = list(filter(antithreshold, items))
  for item in items:
    item.value = int(item.value / (P_g*precision**2))
  dynamic_states = solve_dynamic(expensive_items, max_weight, 2/precision**2+1)
  max_value = 0
  max_value_exp_rounded_value = 0
  max_value_greedy_solution_size = 0
  for rounded_value in range(len(dynamic_states)):
    if dynamic_states[rounded_value].min_weight > max_weight:
      continue
    local_greedy = solve_greedy(
        items=cheap_items,
        max_weight=max_weigth-dynamic_states[rounded_value].min_weight
    )
    if local_greedy.value + dynamic_states[rounded_value].value > max_value:
      max_value = local_greedy.value + dynamic_states[rounded_value].value
      max_value_exp_rounded_value = rounded_value
      max_value_greedy_solution_size = local_greedy.size
  current_value = max_value_exp_rounded_value
  maximum_set_items = []
  while current_value > 0) {
    maximum_set_items.append(
        expensive_items[dynamic_states[current_value].last_item]
    )
    current_value -= maximum_set_items[-1].round_value
  maximum_set_items.reverse()
  for i in range (max_value_greedy_solution_size):
    maximum_set_items.append(cheap_items[i])
  return maximum_set_items

In [0]:
class GeneratorsFactory:

  @staticmethod
  def get_power_of_two_items_set(size, min_value):
    def items_generator():
      nonlocal size, min_value
      for _ in range(size):
        yield min_value
        min_value *= 2
    return wrapped_generator

  @staticmethod
  def get_sequential_items_set(size):
    return lambda: range(1, size+1)

# Задача о рюкзаке  

##### проект №52  


### Выполнил: Михаил Цион, 795




# Основные определения  

> **Определение:** Машина распознаёт язык $A$ за время $T(n)$, если она принимает все слова, лежащие в $A$, отвергает все слова, не лежащие в $A$, и на каждом слове $x$ работает не больше $T (|x|)$ шагов.

> **Определение:** Классом ${\bf DTIME(T (n))}$ называется класс языков, которые распознаются за время $O(T(n))$ на детерминированной машине Тьюринга. Иными словами, время работы машины на любом слове длины $n$ не превосходит некоторой константы, умноженной на $T(n)$.

> **Определение:** Классом ${\bf P}$ называется множество языков, распознающихся за полиномиальное время. Формально он определяется так:
$$P =\bigcup\limits_{c=1}^{\infty} {\bf DTIME(n^c)} $$

> **Определение:** Классом ${\bf NTIME(T(n))}$ называется множество языков, распознаваемых на недетерминированной машине Тьюринга за время $O(T(n)$).

> **Определение:** Классом ${\bf NP}$ называется множество языков $A$, для которых существует функция $V(x, s)$ с булевыми значениями, вычислимая за полиномиальное время от длины первого аргумента, такая что:
*   Если $x \in A$, то $ \exists s \hspace{2pt} V(x, s) = 1$;
*   Если $x \notin A$, то $ \forall s \hspace{2pt} V (x, s) = 0$.


> **Определение:** Пусть $A$ и $B$ суть два языка. Тогда $A$ сводится по Карпу к $B$, если существует всюду определённая функция $f : \{0, 1\}^* \longrightarrow \{0, 1\}^*$ , вычислимая за полиномиальное время, такая что $x \in A \longleftrightarrow f (x) \in B$. Обозначение: $A \leq_p B$ . (Индекс $p$ означает полиномиальность).

> **Определение:** Язык $B$ является ${\bf NP}$**-трудным**, если для любого $A \in {\bf NP}$ выполнено $A \leq_p B$. Язык $B$ является ${\bf NP}$**-полным**, если он ${\bf NP}$**-трудный** и лежит в ${\bf NP}$.

# PROBLEM FORMULATION

Дано:
*   $n \in \mathbb{N}$ предметов
*   $w_{i} > 0 -$ вес $i-$го предмета
*   $c_{i} > 0 -$ cтоимость $i-$го предмета
*   $W \geq 0 -$ вместимость рюкзака  

Требуется найти такое множество предметов $M=\{(w_{j}, c_{j})| j \in \overline{1..n}\} \subseteq \bigcup\limits_{i=1}^{n} {\{(w_{i}, c_{i})\}}$, чтобы
$\sum\limits_{(w, c) \in M} w \leq W$, $\sum\limits_{(w, c) \in M} c \longrightarrow \max$. Допускается пустое множество предметов, т.е. $\forall{i} \in \overline{1..n} \space W < w_{i}$.

Далее, задачу о рюкзаке будем называть просто ${\bf KNAPSACK}$.

# KNAPSACK $\in$ NP-Complete  

---

Прежде, чем доказывать $\textbf{NP}$**-полноту** для $\textbf{KNAPSACK}$, сделаем переформулировку задачи, сведя нашу задачу к задачи распознования языка: к условию добавим константу $P \geq 0$ и будем возвращать **1**, если $\exists \hspace{2pt} I \subset \overline{1..n}$ и выполнено:
*   $\sum\limits_{i \in I} w_i \leq W$
*   $\sum\limits_{i \in I} c_i \geq P$,  
иначе возращаем **0**.

---

### KNAPSACK $\in$ NP  
Для доказательства принадлежности к $\textbf{NP}$ воспользуемся сертификатным определением.  
Возмем сертификат $s-$ поднабор $\overline{1..n}$. Если $\exists{I}$, удовлетворяющий требованиям выше, то $\exists{s}$, который подадим на вход, как второй аргумент $V(\cdot; \cdot)$, а $V(\cdot; s)$ проверит, что $I$ соответствует соотношениям выше.  
   
**Следовательно, KNAPSACK принадлежит NP.**

---

### KNAPSACK $\in$ NP-Hard  
Для начала, рассмотрим частный случай задачи о рюкзаке $-$ $\textbf{SUBSETSUM}$.
Пусть дано множество $S$, состоящее из $n$ чисел. Нужно выяснить, существует ли его подмножество с суммой чисел, равной $s$. Это можно рассматривать как $\textbf{KNAPSACK}$, где $\forall{i}: c_{i} = w_{i}, P = W = s$. Будем обозначать данную задачу как $(S, s)$.

Сведем $\textbf{NP}$-трудную задачу о выполнимости булевой формулы в форме 3-КНФ ($\textbf{3-SAT}$) к $\textbf{SUBSETSUM}$, а т.е. и к $\textbf{KNAPSACK}$.

Пусть задана 3-КНФ формула $\phi$ от $n$ переменных, состоящая из $k$ пар скобок $C_i$. Считаем, что ни одна пара скобок не содержит одновременно переменную и ее отрицание, и каждая переменная входит хотя бы в одну пару скобок.

Построим сводящую функцию $f: \phi \Rightarrow (S, s) $: \\
Для каждой переменной $x_i$ и для каждой пары скобок $C_j$ создадим по два числа в десятичной системе счисления, каждое длиной $n+k$ цифр. Эти числа образуют $S$. Присвоим каждому разряду полученных чисел метку (одинаковую для всех чисел), соответствующую либо переменной, либо паре скобок. Метки, соответствующие парам скобок, присвоены $k$ младшим разрядам чисел.
*  В числе $s$ все разряды, соответствующие переменным, установим 1, а оставшиеся сделаем равными 4.
*  Каждой переменной $x_i$ соответствуют два числа из $S$: $v_i$ и $u_i$. В обоих числах установим разряд, соответствующий данной переменной равным $1$, все остальные разряды переменных -- $0$. Для числа $v_i$ установим разряды, соответствующие скобках, в которые входит $x_i$, равными $1$, все остальные -- $0$. В $u_i$ же сделаем то же самое, но с $\lnot x_i$
*  Каждой паре скобок $C_{j}$ соответствуют два числа из $S$ : $d_{i}$ и $e_{i}$. Оба этих числа содержат 0 во всех разрядах, кроме соответствующего $C_{j}$. В этом разряде у $d_{i}$ 1, а у $e_{i}$ 2.  

Докажем корректность определенной выше функции, т.е. то, что мы действительно сводим $\textbf{3-SAT}$ к $\textbf{SUBSETSUM}$, и делаем это полиномиально.
*  Полученное множество $S$ состоит из $2(n + k)$ чисел, выставить каждый разряд точно возможно за полином, значит и само сведение выполняется за полином.
*  Пусть в $S$ существует подмножество $S'$, т.ч. $\sum\limits_{s_{i} \in S'} s_{i} = s$. Докажем, что $\phi$ выполнима. Заметим, набору $S'$ обязательно принадлежит ровно одно из $v_{i}$ и $u_{i}$, т.к. в $s$ соответствующий $x_{i}$ разряд равен $1$. Если принадлежит $u_{i}$ сделаем $x_{i} = 1$, иначе $-$ $0$. Далее, т.к. разряды, соответствующие скобкам равны $4$, а с помощью $d_{j}$ и $e_{j}$ мы можем получить максимум число $3$ в разряде, то в любой скобке есть хотя бы один выбранный нами терм ($x_{i}$, если в $S'$ лежит $u_{i}$, или его отрицание в другом случае). А значит, каждый дизъюнкт верен. Значит, верна и исходная формула.
*  Пусть $\phi$ выполнима, т.е. существует набор значений переменных $y_{1},...,y_{n}$, при которых формула истинна. Пусть $S$ и $s$ получены после сводимости. Нужно доказать, что в $S$ существует подмножество (обозначим за $S'$) с суммой, равной $s$. Для каждой переменной $x_{i}$ добавим $v_{i}$ в $S'$, если $y_{i} = 1$, иначе $-$ $u_{i}$. Заметим, что для каждого разряда, соответствующего скобках, в уже набранной части $S'$ есть не менее одного и не более трех чисел, у которых в данном разряде стоит $1$. Значит для каждого соответствующего паре скобок $C_{j}$ разряда мы сможем выбрать одно или оба числа $d_{j}$ и $e_{j}$ так, чтобы сумма в данном разряде стала равной $4$. Добавим их в $S'$. Заметим, что суммы во всех разрядах, соответствующих переменным, равны $1$, так как для каждого $i$ выбиралось только одно число из $v_{i}$ и $u_{i}$. Значит, теперь $\sum\limits_{s_{i} \in S'} s_{i} = s$.  

Значит, доказали **NP**-трудность задачи $\textbf{SUBSETSUM}$, а, значит, и **NP**-трудность задачи о рюкзаке. **Следовательно, KNAPSACK принадлежит к NP-трудным.**


---


 Доказав принадлежность к **NP** и **NP-трудным**, мы доказали, что **KNAPSACK принадлежит к NP-полным.**


**Рассмотрим** несколько подходов решения задачи о рюкзаке.

# Bruteforce algorithm  

*Самым тривиальным* решением задачи о рюкзаке есть полный перебор всех подмножеств множества предметов и отыскания того набора, вес которого не превосходит установленного порога, а суммарная стоимость максимальна.  

**Время работы алгоритма:** $O(n \cdot 2^{n})$.


# Implementation

In [0]:
import functools
from itertools import (
  chain,
  combinations)
import math
import operator

In [0]:
class Item(object):

  __slots__ = ['weight', 'cost', 'rounded_cost']

  def __init__(self, weight=0, cost=0, rounded_cost=0, *args, **kwargs):
    self.weight = weight
    self.cost = cost
    self.rounded_cost = rounded_cost

Лямбда-функция $\verb|all_indexes_subsets|$ от $\verb|n|$ возращает список элементов из $2^{\{0...n-1\}}-$ множество множеств всех возможных индексов предметов (в теоретическом изложении индексация начинается с $1$, а в реализации c $0$).

In [0]:
all_indexes_subsets = lambda size: filter(
    lambda subset: len(subset) > 0,
    chain(*[combinations(range(size), i) for i in range(size+1)])
)

In [0]:
def launch_bruteforce(items, max_weight) -> tuple:
  total_value, total_weight = 0, 0
  for index_subset in all_indexes_subsets(len(items)):
    sub_total_value, sub_total_weight = 0, 0
    for index in index_subset:
      sub_total_value += items[index].value
      sub_total_weight += items[index].weight
    if sub_total_value > total_value and sub_total_weight <= max_weight:
      total_value, total_weight = sub_total_value, sub_total_weight
  return total_weight, total_value

# Greedy algorithm  

**Алгоритм:**  
*  Удалим все объекты, вес которых больше, чем $W$. Если для оставшихся $n^{'}\leq n$ объектов выполнено: $\sum\limits_{i=1}^{n^{'}} w_{i} \leq W$, то все эти объекты и являются ответом жадного алгоритма. Иначе переходим к следующим пунктам.
*  Обозначим $p_{i} := \frac{c_{i}}{w_{i}}$ и отсортируем элементы ${x_i}$ по убыванию $p_i$
*  Выберем $k:\space\sum\limits_{i=1}^{k} w_i \leq W < \sum\limits_{i=1}^{k + 1} w_i$.
*  Первые $k$ объектов после сортировки являются ответом жадного алгоритма..  

**Заметим**, что для ответа такого алгоритма выполнено (обозначим за $C'$ значение истинного ответа на задачу):
$$
\sum\limits_{i=1}^{k} c_{i} \leq C' < \sum\limits_{i=1}^{k+1} c_{i}
$$  

**Доказательство:**  
Обозначим $W' = \sum\limits_{i=1}^{k + 1} w_i$.  
Для задачи с максимальным весом $W'$ решением будет будут первые $(k+1)$ объект после сортировки, с общей стоимостью $\sum\limits_{i=1}^{k + 1} c_i$, т.к. эти предметы обладают большей ценой за единицу измерения веса и занимают весь вес. Значит, исходная задача не могла иметь решение с большей стоимостью, т.к. $W \leq W'$.  

**Время работы алгоритма:**  
$O(n \log n)$ или $O(n^{2})$, в зависимости от используемой сортировки.

# Implementation

In [0]:
class GreedySolution(object):
    
    __slots__ = ['size', 'weight', 'cost']

    def __init__(self, size=0, weight=0, cost=0, *args, **kwargs):
        self.size = size
        self.weight = weight
        self.cost = cost

    def __iadd__(self, item: Item):
      self.size += 1
      self.weight = item.weight
      self.cost = item.cost

In [0]:
def launch_greedy(items, max_weight) -> GreedySolution:
  lifting_items = list(filter(lambda item: item.weight < max_weight, items))
  solution = functools.reduce(function=operator.iadd, iterable=lifting_items,
                        initializer=GreedySolution())
  if solution.weight > max_weight:
    lifting_items.sort(key=lambda item: -item.cost/item.weight)
    solution = GreedySolution()
    while solution.weight <= max_weight and solution.size < len(lifting_items):
      if solution.weight + lifting_items[solution.size] <= max_weight:
        solution += lifting_items[solution.size]
      else:
        break
  return solution

# Pseudo-polynomial algorithm  

В полиномиальной схеме приближения задачи о рюкзаке будет использоваться псевдополиномиальный алгоритм c целыми стоимостями.
> **Определение:** Алгоритм называется псевдополиномиальным, если работает полиномиально от размера входных данных, записанных в унарном виде.

Обозначим за $w(k, c)$ наименьшей вес подмножества первых $k$ объектов, имеющего стоимость $c$. Имеют место следующие соотношения для нашего алгоритма:
*  $w(0, 0) = 0$
*  $w(0, c) = +\infty$
*  $w(k+1, c) = \min (w(k, c), \hspace{2pt} w(k, c - c_{k+1}) + w_{k + 1})$

Нетрудно заметить, что посторенный алгоритм относится к алгоритмам из двумерной динамики.

**Используемая память:**  
Пусть мы обладаем знанием, что максимальная стоимость $C' \leq T$, тогда всего потребуется $(n+1)T$ состояний, т.е. $O(T)$ памяти.  

**Время работы алгоритма:**  
Работает же алгоритм за $O(n \cdot T)$, т.к. при добавлении каждого объекта достаточно просмотреть $T$ состояний. Понятно, что $T$ не больше $\sum_{i=1}^{n}c_{i} \leq n\cdot \max\limits_{1\leq i\leq n} c_{i}$. Следовательно, итоговое время работы полиномиально.

In [0]:
class DynamicState(object):
  
  __slots__ = ['min_weight', 'value', 'last_item']

  def __init__(self, min_weight=math.inf, value=0, last_item=-1, *args, **kwargs):
    self.min_weight = min_weight
    self.value = value
    self.last_item = last_item

In [0]:
def launch_dynamic(items, max_weight, max_cost=0) -> list:
  if max_cost == 0:
    max_cost = functools.reduce(
        function=lambda total, item: total + item.cost,
        iterable=items)
  states = [DynamicState() for _ in range(max_cost+1)]
  states[0].min_weight = 0
  for (index, item) in enumerate(items):
    for cost in range(max_cost, item.cost, -1):
      if item.weight + states[cost-item.cost].min_weight < \
      states[cost].min_weight:
        states[cost] = DynamicState(
            item.weight + states[cost-item.cost].min_weight,
            item.cost + states[cost-item.cost].cost, index
        )
  return states

# Polynomial approximation  

> **Определение:** Пусть у нас есть $\textbf{NP}$-сложная задача максимизации функции $f$, $E-$ ее входные данные, $C_{E}-$ оптимальный ответ на входе $E$. Алгоритм называется **$(1 - \varepsilon)-$приближением** для решения данной задачи, если на входе $(E, \varepsilon)$ он выдает такое ответ $C_{E}^{'}$, что $C_{E}^{'} \geq (1 - \varepsilon) \cdot C_{E}$. Если алгоритм работает за полиномиальное время от размера входа и $\frac{1}{\varepsilon}$, то он называется **полиномиальным приближением**.  

В предположении $\textbf{P} \neq \textbf{NP}$ задача о рюкзаке не имеет полиномиального алгоритма решения. Но при этом, если придумать алгоритм полиномиального приближения, то за полиномиальное время мы научимся приближенно решать задачу, получая ответ заданной точности.  

Обозначим максимально возможную стоимость предметов, общего веса $\leq W$ за $C'$. Рассмотрим два алгоритма:  


*   основанный на подходе динамического программирования
*   алгоритм Ибарры-Кима



## Polynomial approximation by dynamic programming  

**Алгоритм:**
*  Удалим все предметы, масса которых больше максимально возможной.
*  Определим $c_{0} := \max\limits_{1\leq i\leq n} c_{i}$. 
*  Заменим стоимость каждого предмета $c_{i}$ на $c_{i}^{'} := \left[\frac{c_{i}}{\delta}\right]$, где нормирующий коэффициент $\delta$ равен $\frac{\varepsilon \cdot c_{0}}{n}$, а $n-$ это как и всегда количество предметов. 
*  Для новых стоимостей найдем наилучшее решение псевдополиномиальным алгоритмом, предметы из этого решения и будут ответом (обозначим их индексы за $I$).  

> **Теорема.** Построенный алгоритм является полиномиальным приближения задачи о рюкзаке. Временная сложность алгоритма $O(\frac{n^{3}}{\varepsilon})$, пространственная сложность $O(\frac{n^{2}}{\varepsilon})$.  

> **Доказательство.** Докажем, что алгоритм действительно строит $(1 - \varepsilon)-$приближение. Мы знаем, что псевдополиномиальное приближение находит точное решение. Т.е. переходя от начальных нецелых стоимостей к целым, путем взятия целой части при делении на $\delta$, мы найдем точное решение в стоимостях $c^{'}$. Пусть $I_{0}-$индексы предметов из истинного решения исходной задачи, $I-$индексы решения задачи в стоимостях $c^{'}>0$, тогда верно:
$$
\sum\limits_{i \in I} \frac{c_{i}}{\delta} \geq \sum\limits_{i \in I} \left[\frac{c_{i}}{\delta}\right] \geq \sum\limits_{i \in I} c_{i}^{'} \geq \sum\limits_{j \in I_{0}} c_{j}^{'}
.$$
Значит,
$$
\sum\limits_{i \in I} c_{i} \geq \delta \cdot \sum\limits_{j \in I_{0}} c_{j}^{'}
.$$
И из неравенства:
$$
\sum\limits_{i \in I_{0}} (c_{i} - \delta \cdot c_{i}^{'}) = \sum\limits_{i \in I_{0}} (c_{i} - \delta \cdot \left[\frac{c_{i}}{\delta}\right]) = \delta\cdot\sum\limits_{i \in I_{0}} (\frac{c_{i}}{\delta} - \left[\frac{c_{i}}{\delta}\right]) \leq \delta\cdot\sum\limits_{i \in I_{0}} 1 = \delta \cdot |I_{0}| \leq \delta \cdot n = \frac{\varepsilon \cdot c_{0}}{n} \cdot n \leq \varepsilon \cdot c_{0} \leq \varepsilon \cdot C^{'}
.$$
Получаем оценку:
$$
\sum\limits_{i \in I} c_{i}\geq \delta \cdot \sum\limits_{j \in I_{0}} c_{j}^{'} \geq \sum\limits_{i \in I_{0}} c_{i} - \varepsilon \cdot C^{'} = (\frac{\sum\limits_{i \in I_{0}} c_{i}}{C^{'}} - \varepsilon) \cdot C^{'} \geq (1 - \varepsilon) \cdot C^{'}
,$$
т.к. $\sum\limits_{i \in I_{0}} c_{i} \geq C^{'}.$  
А, значит, найденный нами набор является $(1-\varepsilon)-$приближением ответа. Заметим, т.к. $\frac{c_{0}}{\delta} = \frac{n}{\varepsilon}$, то максимальная стоимость решения не больше, чем $\frac{n^{2}}{\varepsilon}$, следовательно, и алгоритм работает за $O(\frac{n^{3}}{\varepsilon})$. При этом пространственная сложность $O(\frac{n^{2}}{\varepsilon})$


## Implementation

## Polynomial approximation by Ibarra-Kim  

Алгоритм гарантированно получает $(1-\varepsilon)-$приближение в случае, когда $C' \geq \max{\{\sum\limits_{i=1}^{k} c_{i}; c_{k+1}\}}$, где предметы отсортированны как в жадном алгоритме, $k$ взято из него же. Для этого случая будет доказана его корректность.  
Суть алгоритма состоит в разделении предметов на **дорогие** и **дешевые**. Для дорогих предметов воспользуемся псевдополиномиальным алгоритмом. Для дешевых, учитывая оставшийся вес от выбранных дорогих, воспользуемся жадным алгоритмом.  

**Алгоритм**:
*  Обозначим ${\bf C_{m}} = \frac{1}{2} \cdot \sum\limits_{i=1}^{k+1} c_{i}$, где $k$ взято из жадного алгоритма (перед этим отсортировали предметы по $p_{i}=\frac{c_{i}}{w_{i}})$. Имеем ${\bf C_{m}} \leq C' \leq 2 \cdot {\bf C_{m}}$, при условии, что $C' \geq \max{\{\sum\limits_{i=1}^{k} c_i; c_{k + 1}\}}$.
*  Разделим предметы **дорогие** и **дешевые**. Если $c_i \geq \varepsilon \cdot {\bf C_m} $, то он дорогой, иначе $-$ дешевый. Пусть дорогие объекты имеют индексы: $1,...,l$, дешевые $-$ $l+1,...,n$.
*  Обозначим $\delta := \varepsilon^2 \cdot {\bf C_{m}}$. Заменим стоимость каждого дорогого на целое число: $c_{i} \rightarrow \left[\frac{c_i}{\delta}\right]$
*  Обозначим $C_{lim} := [\frac{2 \cdot {\bf C_{m}}}{\delta}]$, где $m-$ количество дорогих элементов. 
*  Далее $\forall c \in \{0,...,C_{lim}\}$: для дорогих $-$ псевдополиномиальным алгоритмом находим $w(m, c)$ (мы нашли минимальный вес дорогих имеющих стоимость не меньше $c$), обозначим соответствующую им стоимость за $C_{c}^{1}$; дешевых $-$ жадным алгоритмом находим объекты, дающие максимальную стоимость, имеющих вес не больше $(W - w(m, c))$, обозначим соответствующую им стоимость за $C_{c}^{2}$; суммируем найденные стоимости.
*  Нашим ответом будет $\max\limits_{0\leq j\leq C_{lim}} \{C_{j}^{1} + C_{j}^{2}\}$.   

> **Теорема.** При условии $C' \geq \max{\{\sum\limits_{i=1}^{k} c_{i}; c_{k + 1}\}}$ данный алгоритм строит $(1-\varepsilon)-$приближение истинного решения. Временная сложность алгоритма $O(n \cdot \log n + \frac{n}{\varepsilon^2})$.  

> **Доказательство.**  Докажем, что алгоритм строит $(1-\varepsilon)-$приближение.  
Для **дорогих** предметов верно неравенство, доказанное ранне (в силу того, что целая часть от частного, умноженная на делитель, отличается от делимого менее, чем на делитель):
$$
c_{i} - \delta \cdot c_{i}^{'} \leq c_{i} - \delta \cdot \left[\frac{c_{i}}{\delta}\right] < \delta
$$
При этом, $\delta = \varepsilon^2 \cdot {\bf C_{m}}$, и для любого дорогого предмета выполнено: $c_{i} \geq \varepsilon \cdot {\bf C_{m}}$. Из этого получаем, что:
$$
c_{i} - \delta c_{i}^{'} < \varepsilon^2 \cdot {\bf C_{m}} \leq \varepsilon^2 \cdot \frac{c_{i}}{\varepsilon} = \varepsilon \cdot c_{i}
$$
То есть суммарная ошибка для дорогих не больше, чем $2 \cdot \varepsilon \cdot {\bf C_{m}}$.  
Для **дешевых** предметов мы применяем жадный алгоритм, значит, верно, что найденное решение отличается от истинного не более, чем, на $c_{l+1}$, где $l-$ размер множества предметов, полученного жадным алгоритмом для дешевых предметов, т.е. не более, чем на $\varepsilon \cdot {\bf C_{m}}$. Значит, суммарная ошибка на всех предметах не более $3 \cdot \varepsilon \cdot {\bf C_{m}} \leq 3 \cdot \varepsilon \cdot C^{'}$.  
Осталось доказать, что алгоритм работает за  $O(n \cdot \log n + \frac{n}{\varepsilon^2})$.  
Сложность сортировки, поиска ${\bf C_{m}}$ и разделения предметов составляет $O(n \cdot \log n)$.  
Сложность подсчёта $w(n, c)$ составляет $O({\bf C_m} \cdot \frac{n}{\delta}) = O(\frac{n}{\varepsilon^{2}})$.  
Тогда общая **сложность алгоритма**: $O(n \cdot \log n + \frac{n}{\varepsilon^2})$.

## Implementation

# Testing  



In [0]:
import random


class GeneratorsFactory:

  @staticmethod
  def get_power_of_two_items_set(size, start_weight=1):
    def items_generator():
      nonlocal size, start_weight
      for _ in range(size):
        yield Item(start_weight, start_weight)
        start_weight *= 2
    return wrapped_generator

  @staticmethod
  def get_sequential_items_set(size, start_weight=1):
    for value in range(start_weight, size+start_weight):
      yield Item(value, value)

  @staticmethod
  def get_random_items_set(size, lrange=1, rrange=100):
    for value in range(start_weight, size+start_weight):
      yield Item(weight=random.randint(lrange, rrange), 
                 cost=random.randint(lrange, rrange))